---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import datetime

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [9]:
def get_list_of_university_towns():
    unitowns = pd.read_table('university_towns.txt', sep='\n', header=None, names=['RegionName'])
    unitowns['State'] = unitowns[unitowns['RegionName'].str.contains('\[edit\]')]
    unitowns = unitowns.fillna(method='pad')
    unitowns.drop(unitowns[unitowns['RegionName'].str.contains('\[edit\]')].index, inplace=True)
    unitowns['State'] = unitowns['State'].str.replace('\[.*\]','').str.strip()
    unitowns['RegionName'] = unitowns['RegionName'].str.replace('\(.*','').str.replace('\[.*\]','').str.strip()
    unitowns = unitowns[['State','RegionName']].reset_index(drop=True)
    return unitowns

In [14]:
gdp = pd.read_excel('gdplev.xls', header=None, skiprows=8, usecols=[4,6], names=['Duration','GDP'])
gdp = gdp[gdp['Duration']>'2000'].reset_index(drop=True)

In [21]:
def get_recession_start():
    for quarter in range(len(gdp)-2):
        if (gdp['GDP'][quarter] > gdp['GDP'][quarter+1]) & (gdp['GDP'][quarter+1] > gdp['GDP'][quarter+2]):
            return gdp['Duration'][quarter+1]

In [22]:
def get_recession_end():
    for quarter in range(len(gdp)-2):
        if (gdp['GDP'][quarter] < gdp['GDP'][quarter+1]) & (gdp['GDP'][quarter+1] < gdp['GDP'][quarter+2]) & (gdp['Duration'][quarter] > get_recession_start()):
            return gdp['Duration'][quarter+2]

In [23]:
def get_recession_bottom():
    temp = gdp[gdp['Duration'].between(get_recession_start(),get_recession_end(),inclusive=True)]
    return temp['Duration'][temp['GDP'].idxmin()]

In [3]:
def convert_housing_data_to_quarters():
    zillow = pd.read_csv('City_Zhvi_AllHomes.csv')
    zillow['State'] = zillow['State'].map(states)
    zillow.set_index(['State','RegionName'], inplace=True)
    zillow = zillow[zillow.columns[49:]]
    zillow.columns = zillow.columns.astype('datetime64[ns]')
    housing = zillow.groupby(pd.PeriodIndex(zillow.columns, freq='Q'), axis=1).mean()
    housing.columns = housing.columns.to_series().astype(str)
    housing.columns = map(str.lower, housing.columns)
    return housing

In [1]:
def run_ttest():
    df = convert_housing_data_to_quarters()
    df = df[df.columns[df.columns.tolist().index(get_recession_start())-1]]/df[get_recession_bottom()]
    tup = [tuple(x) for x in get_list_of_university_towns().values]
    df1 = df[df.index.isin(tup)]
    df2 = df[~df.index.isin(tup)]

    test = ttest_ind(df1,df2,nan_policy='omit')
    test.pvalue
    if test.pvalue < 0.01:
        different=True
    else:
        different==False
    if df1.mean()<df2.mean():
        better = 'university town'
    else:
        better = 'non-university town'
    return (different,test.pvalue,better)